In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.models import Model 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.applications import InceptionResNetV2 
! pip install split-folders
import tensorflow as tf
import split_folders
from keras import optimizers

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras import optimizers

In [ ]:
train_data_dir = "/content/drive/My Drive/Neural_ProjectC2/augmented_images/train"
val_data_dir = "/content/drive/My Drive/Neural_ProjectC2/augmented_images/validation"
num_train = 0
num_val = 0

for i in os.listdir(train_data_dir):
  if i.startswith("."):
    os.rmdir(os.path.join(train_data_dir, i))
  else:
    num_train += len(os.listdir(os.path.join(train_data_dir, i)))
print("train=",num_train) 

for i in os.listdir(val_data_dir):
  if i.startswith("."):
    os.rmdir(os.path.join(val_data_dir, i))
  else:
    num_val += len(os.listdir(os.path.join(val_data_dir, i)))
print("val=",num_val)

img_width, img_height = 224, 224
# Check if the images are RGB and change the channels likewise
if K.image_data_format() == 'channels_first':
  input_shape= (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense
base_model =  DenseNet121(input_shape=(224, 224, 3),
                                  weights='imagenet',
                                  include_top = False,
                                  pooling='avg')

for layer in base_model.layers:
  layer.trainable = False

x = base_model.output
x = Dense(1024)(x)


x = Activation('relu')(x)
x = Dense(512)(x)


x = Activation('relu')(x)
predictions = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(), metrics = ['accuracy'])

model.summary()



In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

def get_callbacks():

   path_checkpoint ='checkpoint_keras'  
   log_dir='logs'
   
   callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                         monitor='val_loss',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         period=1)
   
   callback_early_stopping = EarlyStopping(monitor='val_loss',
                                           patience=5,
                                           verbose=1)
   
   callback_tensorboard = TensorBoard(log_dir=log_dir,
                                      histogram_freq=0,
                                      write_graph=False)
   
   callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.1,
                                          min_lr=1e-4,
                                          patience=3,
                                          verbose=1)

   callbacks = [callback_checkpoint, callback_tensorboard, callback_reduce_lr]

   return callbacks

In [ ]:
batch_size = 32
epochs = 50

train_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                    target_size =(img_width, img_height), 
                                                    batch_size = batch_size, class_mode = 'categorical')

validation_generator = train_datagen.flow_from_directory( val_data_dir, 
                                                         target_size =(img_width, img_height), 
                                                         batch_size = batch_size, class_mode ='categorical') 

history=model.fit_generator(train_generator, 
                    steps_per_epoch = num_train // batch_size, 
                    epochs = epochs, validation_data = validation_generator, 
                    validation_steps = num_val// batch_size,
                    workers=1,use_multiprocessing= False,callbacks = get_callbacks())



In [ ]:
model.save("/content/drive/My Drive/Neural_ProjectC2/models/model_dense121.h5")

In [ ]:
import numpy
import keras
import sklearn.metrics as metrics
img_width, img_height = 224, 224
batch_size = 8
epochs = 100
test_set_dir = "/content/drive/My Drive/Neural_ProjectC2/data_test"

num_test = len(os.listdir(test_set_dir))

print ("Number of images in test set: ", num_test)

#model = get_model(0.0001)

model = keras.models.load_model("/content/drive/My Drive/Neural_ProjectC2/models/model_dense121.h5")
  
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_set_dir, target_size =(img_width, img_height), 
                                                  batch_size = batch_size, class_mode =None, shuffle = False) 
#print(test_datagen)
test_steps_per_epoch = test_generator.n/test_generator.batch_size
#print(test_generator.n)
test_generator.reset()
predictions = model.predict_generator(test_generator, steps = test_steps_per_epoch)

print("PREDICTIONS: --->")
print(predictions)

predicted_classes = numpy.argmax(predictions, axis=1)

print("PREDICTED CLASSES: --->")
print (predicted_classes)

In [ ]:


from matplotlib import pyplot as plt
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
from keras.utils import to_categorical

# one hot encode
encoded = to_categorical(predicted_classes)
print(encoded)

In [ ]:
np.savetxt("/content/drive/My Drive/Neural_ProjectC2/predict.csv", encoded, delimiter=',',fmt='%d')